In [1]:
import numpy as np
import matplotlib.pyplot as plt

Training set

In [2]:
T = 50
Nt = 50
dt = T/Nt
x0 = 1
y0 = 0.1
rho_v = np.arange(0.0, 0.1, 0.01)
alpha_v = np.arange(0.0, 0.1, 0.01)
beta_v = np.arange(0.0, 0.1, 0.01)
gamma_v = np.arange(0.0, 0.1, 0.01)
delta_v = np.arange(0.0, 0.1, 0.01)
n_tot = 4**5
k=0
input_list = []
output_list =[]

for rho in rho_v:
    for alpha in alpha_v:
        for beta in beta_v:
            for gamma in gamma_v:
                for delta in delta_v:
                    k += 1
                    # print('Advancing: '+str(k/n_tot*100)+'%')
                    x_ex = np.zeros((Nt+1,2))
                    tt=np.arange(0,T+dt,dt)
                    for i,t in enumerate(tt):
                        if i==0:
                            x_ex[i,:] = np.array([x0,y0])
                        else:
                            x_ex[i,0] = x_ex[i-1,0] + dt*x_ex[i-1,0]*(alpha-beta*x_ex[i-1,1]-rho*x_ex[i-1,0])
                            x_ex[i,1] = x_ex[i-1,1] + dt*x_ex[i-1,1]*(-gamma+delta*x_ex[i-1,0])
                            input_list.append(x_ex[i-1,:])
                            output_list.append(x_ex[i,:])

input_train = np.array(input_list)
output_train = np.array(output_list)


In [3]:
plot_Flag = False
# plot_Flag = True
if plot_Flag:
    np.save('../data/input_train.npy', input_train)
    np.save('../data/output_train.npy', output_train)

Test set

In [2]:
T = 50
Nt = 50
dt = T/Nt
x0 = 1
y0 = 0.1
rho_v = np.arange(0.005, 0.09, 0.01)
alpha_v = np.arange(0.005, 0.09, 0.01)
beta_v = np.arange(0.005, 0.09, 0.01)
gamma_v = np.arange(0.005, 0.09, 0.01)
delta_v = np.arange(0.005, 0.09, 0.01)
n_tot = rho_v.shape[0]**5
k=0
input_list = []
output_list =[]

for rho in rho_v:
    for alpha in alpha_v:
        for beta in beta_v:
            for gamma in gamma_v:
                for delta in delta_v:
                    k += 1
                    # print('Advancing: '+str(k/n_tot*100)+'%')
                    x_ex = np.zeros((Nt+1,2))
                    tt=np.arange(0,T+dt,dt)
                    for i,t in enumerate(tt):
                        if i==0:
                            x_ex[i,:] = np.array([x0,y0])
                        else:
                            x_ex[i,0] = x_ex[i-1,0] + dt*x_ex[i-1,0]*(alpha-beta*x_ex[i-1,1]-rho*x_ex[i-1,0])
                            x_ex[i,1] = x_ex[i-1,1] + dt*x_ex[i-1,1]*(-gamma+delta*x_ex[i-1,0])
                            input_list.append(x_ex[i-1,:])
                            output_list.append(x_ex[i,:])

input_test = np.array(input_list)
output_test = np.array(output_list)


In [3]:
plot_Flag = False
plot_Flag = True
if plot_Flag:
    np.save('../data/input_test.npy', input_test)
    np.save('../data/output_test.npy', output_test)

In [ ]:
import torch
from pathlib import Path

from neuralop.utils import UnitGaussianNormalizer
from neuralop.datasets.tensor_dataset import TensorDataset
from neuralop.datasets.transforms import PositionalEmbedding


def load_dataset(train_path, test_path,
                batch_size, test_batch_sizes,
                grid_boundaries=[[0,1],[0,1]],
                positional_encoding=False,
                encode_input=False,
                encode_output=False,
                encoding='pixel-wise', 
                channel_dim=1):
    """Loads a small Darcy-Flow dataset
    
    Training contains 1000 samples in resolution 16x16. 
    Testing contains 100 samples at resolution 16x16 and
    50 samples at resolution 32x32.

    Parameters
    ----------
    n_train : int
    n_tests : int
    batch_size : int
    test_batch_sizes : int list
    test_resolutions : int list, default is [16, 32],
    grid_boundaries : int list, default is [[0,1],[0,1]],
    positional_encoding : bool, default is True
    encode_input : bool, default is False
    encode_output : bool, default is True
    encoding : 'channel-wise'
    channel_dim : int, default is 1
        where to put the channel dimension, defaults size is batch, channel, height, width

    Returns
    -------
    training_dataloader, testing_dataloaders

    training_dataloader : torch DataLoader
    testing_dataloaders : dict (key: DataLoader)
    """
    # for res in test_resolutions:
    #     if res not in [16, 32]:
    #         raise ValueError(f'Only 32 and 64 are supported for test resolution, but got {test_resolutions=}')
    # path = Path(__file__).resolve().parent.joinpath('data')
    return load_dataset_pt(train_path, test_path, 
                         batch_size=batch_size, test_batch_size=test_batch_sizes,
                         grid_boundaries=grid_boundaries,
                         positional_encoding=positional_encoding,
                         encode_input=encode_input,
                         encode_output=encode_output,
                         encoding=encoding,
                         channel_dim=channel_dim)

def load_dataset_pt(train_data_path, test_data_path,
                batch_size, test_batch_size,
                grid_boundaries=[[0,1],[0,1]],
                positional_encoding=False,
                encode_input=False,
                encode_output=True,
                encoding='channel-wise', 
                channel_dim=1):
    """Load the Navier-Stokes dataset
    """
    data = np.load(train_data_path)
    # n_train = data.shape[0]
    x_train = data[:, 0]
    y_train = data[:, 1]
    del data

    data = np.load(test_data_path)
    # n_test = data.shape[0]
    x_test = data[:, 0]
    y_test = data[:, 1]
    del data
    
    # if encode_input:
    #     if encoding == 'channel-wise':
    #         reduce_dims = list(range(x_train.ndim))
    #     elif encoding == 'pixel-wise':
    #         reduce_dims = [0]

    #     input_encoder = UnitGaussianNormalizer(x_train, reduce_dim=reduce_dims)
    #     x_train = input_encoder.encode(x_train)
    #     x_test = input_encoder.encode(x_test.contiguous())
    # else:
    #     input_encoder = None

    # if encode_output:
    #     if encoding == 'channel-wise':
    #         reduce_dims = list(range(y_train.ndim))
    #     elif encoding == 'pixel-wise':
    #         reduce_dims = [0]

    #     output_encoder = UnitGaussianNormalizer(y_train, reduce_dim=reduce_dims)
    #     y_train = output_encoder.encode(y_train)
    # else:
    #     output_encoder = None
    output_encoder = None

    train_db = TensorDataset(x_train, y_train, transform_x=PositionalEmbedding(grid_boundaries, 0) if positional_encoding else None)
    train_loader = torch.utils.data.DataLoader(train_db,
                                            batch_size=batch_size, shuffle=True,
                                            num_workers=0, pin_memory=True, persistent_workers=False)

    test_db = TensorDataset(x_test, y_test,transform_x=PositionalEmbedding(grid_boundaries, 0) if positional_encoding else None)
    test_loader = torch.utils.data.DataLoader(test_db,
                                              batch_size=test_batch_size, shuffle=False,
                                              num_workers=0, pin_memory=True, persistent_workers=False)
    test_loaders =  {'1': test_loader}
    # for ( n_test, test_batch_size) in zip(n_test, test_batch_sizes):
    #     print(f'Loading test db with {n_test} samples and batch-size={test_batch_size}')
    #     data = torch.load(Path(data_path).joinpath(f'darcy_test_{res}.pt').as_posix())
    #     x_test = data['x'][:n_test, :, :].unsqueeze(channel_dim).type(torch.float32).clone()
    #     y_test = data['y'][:n_test, :, :].unsqueeze(channel_dim).clone()
    #     del data 
    #     if input_encoder is not None:
    #         x_test = input_encoder.encode(x_test)

    #     test_db = TensorDataset(x_test, y_test, transform_x=PositionalEmbedding(grid_boundaries, 0) if positional_encoding else None)
    #     test_loader = torch.utils.data.DataLoader(test_db,
    #                                               batch_size=test_batch_size, shuffle=False,
    #                                               num_workers=0, pin_memory=True, persistent_workers=False)
    #     test_loaders[res] = test_loader

    return train_loader, test_loaders, output_encoder
